## 175 Project

Setup instructions in `src/README.md`

This code will take about 20 seconds to run once you have all the dependencies installed. There is no data required as all environments are provided by cleanrl and ale-py.

## Imports

In [1]:
import torch
import torch.nn as nn
import gymnasium as gym

from src.cleanrl_utils.evals.dqn_eval import evaluate
from src.cleanrl.dqn_atari import QNetwork, make_env

/home/omar/.cache/pypoetry/virtualenvs/cleanrl-m8vVMvIA-py3.9/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


## Initialize parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env_id = "SpaceInvadersNoFrameskip-v4"
envs = gym.vector.SyncVectorEnv([make_env(env_id, 0, 0, False, '')])

model = QNetwork(envs).to(device)
model_path = "models/0.5_sparsity_dqn.cleanrl_model"
model.load_state_dict(torch.load(model_path, map_location=device))
print(f'Device: {device}')

Device: cpu


## Check Sparsity

In [42]:
weights = [layer.weight if isinstance(layer, nn.Linear) else torch.tensor([0]) for layer in model.network ]
zero_weights = sum([torch.sum(torch.eq(weight, 0)).item() for weight in weights])
nonzero_weights = sum([torch.count_nonzero(weight).item() for weight in weights])

print('Zero weights:', zero_weights) 
print('Nonzero weights:', nonzero_weights)
print('Total sparsity:', zero_weights/(zero_weights+nonzero_weights))

Zero weights: 830160
Nonzero weights: 778552
Total sparsity: 0.5160401613216039


## Evaluate Model (DQN 0.5 sparsity)

In [43]:
episodic_returns = evaluate(
    model_path,
    make_env,
    env_id,
    eval_episodes=5,
    run_name="dqn_eval",
    Model=QNetwork,
    device=device,
    epsilon=0.05,
)

/home/omar/.cache/pypoetry/virtualenvs/cleanrl-m8vVMvIA-py3.9/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-0.mp4.
Moviepy - Writing video /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-0.mp4
eval_episode=0, episodic_return=[465.]
Moviepy - Building video /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-1.mp4.
Moviepy - Writing video /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/omar/Documents/Coding/175/project/videos/dqn_eval/rl-video-episode-1.mp4
eval_episode=1, episodic_return=[435.]
eval_episode=2, episodic_return=[350.]
eval_episode=3, episodic_return=[700.]
eval_episode=4, episodic_return=[470.]


In [44]:
print(f'Mean episodic return of 0.5 sparsity dqn: {sum(episodic_returns)/len(episodic_returns)}')

Mean episodic return of 0.5 sparsity dqn: [484.]


## Video of Model

This code also save a video to the `videos/` folder. Here is an example of what that video will look like:

<video width="640" height="480" controls>
    <source src="example_video.mp4" type="video/mp4">
    Your browser does not support the video tag.
</video>